In [1]:
import pandas as pd
import numpy as np
import operator
import ast
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
import spacy
import re

In [151]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [3]:
print(data['is_question_bad'][0])

0.0


In [4]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [5]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [6]:
first_doc[294:297]

'19 '

In [7]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [8]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [152]:
# Remove uneeded columns
df = data[['question','validated_answers','story_text']]
df = df.dropna()
df = df.iloc[0:100] # reduce size for development purposes

df.head()

,question,validated_answers,story_text
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake..."


In [198]:
# loops through the rows and prints the question along with the first answer given
start_truth = []
end_truth = []
for i, row in df.iterrows():
    try:
        answers = ast.literal_eval(row['validated_answers'])
        sorted_ans = sorted(answers.items(), key=operator.itemgetter(1), reverse=True)
        #print(sorted_ans)
        start, end = sorted_ans[0][0].split(':')
        start_truth.append(int(start))
        end_truth.append(int(end))
    except ValueError:
        start_truth.append(np.nan)
        end_truth.append(np.nan)
        pass
df['start_truth'] = start_truth
df['end_truth'] = end_truth
df = df.dropna()
print(len(df))
df.head()

80


,question,validated_answers,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort...",294,297
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...,103,127
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688,791
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...,301,324
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake...",63,97


In [206]:
final = df.drop(columns=['validated_answers'])
final.head()

,question,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...",294,297
2,who did say South Africa did not issue a visa ...,Johannesburg (CNN) -- Miffed by a visa delay t...,103,127
4,What frightened the families?,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688,791
6,Who is hiring?,CNN affiliates report on where job seekers are...,301,324
8,Iran criticizes who?,"TEHRAN, Iran (CNN) -- Iran's parliament speake...",63,97


In [207]:
full_text = ''
m = 0
for col in ['question','story_text']:
    for text in final[col]:
        m = max(m, len(text))
        full_text += text.lower()
    print(m)
    m=0

# get the set of all characters
characters = tuple(set(full_text))
characters[:5]

68
7361


('a', 'h', 'm', '(', 'b')

In [208]:
# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}
print(char2int)


{'a': 0, 'h': 1, 'm': 2, '(': 3, 'b': 4, '\n': 5, 'y': 6, ' ': 7, '.': 8, 'g': 9, '4': 10, '2': 11, '6': 12, '-': 13, '$': 14, 'q': 15, 'v': 16, '5': 17, 'ñ': 18, 'c': 19, '*': 20, '3': 21, '8': 22, ':': 23, '[': 24, '_': 25, '7': 26, ',': 27, 'd': 28, '9': 29, 'â': 30, 'o': 31, 'j': 32, '"': 33, '!': 34, ']': 35, 'w': 36, 'u': 37, "'": 38, '1': 39, '0': 40, 't': 41, 's': 42, 'f': 43, 'é': 44, 'ã': 45, 'n': 46, 'e': 47, 'z': 48, 'ë': 49, '&': 50, 'l': 51, 'r': 52, 'p': 53, '•': 54, ')': 55, '/': 56, '?': 57, 'i': 58, ';': 59, 'k': 60, 'x': 61, '»': 62, '+': 63, '©': 64, 'ú': 65}


In [209]:
q_max = 68
story_max = 7361

In [210]:
padding_start = False

for i, row in final.iterrows():
    for col, N in zip(['question','story_text'], [q_max, story_max]):
        enc = np.zeros(N, dtype=np.int32)
        
        row[col] = np.array([char2int[char] for char in row[col].lower()])
        l = min(N, len(row[col]))
        if padding_start:
            enc[:l] = row[col][:l]
        else:
            enc[N-l:] = row[col][:l]
        final[col][i] = enc
final.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,question,story_text,start_truth,end_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",294,297
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",103,127
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",688,791
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",301,324
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",63,97


## Train/Test Split

In [211]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final[['question', 'story_text']], 
                                                    final[['start_truth','end_truth']], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [212]:
print(X_train.index)
X_train.head()

Int64Index([ 85, 170, 118, 148,  31, 112, 152, 226,  13, 138, 146,  39, 187,
             54, 120,  22, 223,   6,  35, 109,  15, 183,  11,  94, 213, 159,
            126, 225,  26,  66, 116,  63, 132,  48, 119, 224, 154,  18,  80,
            220, 160, 174, 196, 105,  69,   2, 144,  42,   4,  47, 219,  40,
            161, 211,  25, 139],
           dtype='int64')


,question,story_text
85,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
170,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [213]:
y_train.head()

,start_truth,end_truth
85,2601,2606
170,1250,1308
118,64,98
148,1765,1777
31,1873,1907


In [214]:
class NewsQADataset(Dataset):
    def __init__(self, X, y, N=400, padding_start=False):
        self.y = y.values
        self.question = X['question'].values
        self.story_text =X['story_text'].values
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.question[idx], self.story_text[idx], self.y[idx]

In [215]:
train_ds = NewsQADataset(X_train, y_train)
test_ds = NewsQADataset(X_test, y_test)

In [216]:

train_ds[0]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 58, 46,  7,
        36,  1,  0, 41,  7,  6, 47,  0, 52,  7, 28, 58, 28,  7,  4,  2, 36,
         7, 47, 46, 41, 47, 52,  7, 41,  1, 47,  7, 42, 53, 31, 52, 41, 57],
       dtype=int32),
 array([ 0,  0,  0, ...,  0, 28,  8], dtype=int32),
 array([2601, 2606]))

In [217]:
batch_size = 8
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [219]:
x1, x2, y = next(iter(train_dl))
print(x1.shape, x2.shape, y.shape)
x1, x2, y

torch.Size([8, 68]) torch.Size([8, 7361]) torch.Size([8, 2])


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 36,  1,  0,
          41,  7, 36,  0,  6,  7,  0, 52, 47,  7, 41, 47,  0,  2, 42,  7,  4, 47,
          58, 46,  9,  7, 53, 52, 47, 53,  0, 52, 47, 28,  7, 43, 31, 52,  7, 43,
           0,  2, 58, 51,  0, 52,  7, 19, 58, 52, 19, 37, 58, 41],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0, 36,  1,  0, 41,  7, 36, 58, 51, 51,  7, 46, 47, 47, 28,
           7, 19, 52, 47,  0, 41, 58, 16, 47,  7, 28, 58, 53, 51, 31,  2,  0, 19,
           6,  7,  0, 46, 28,  7, 19, 31, 37, 52,  0,  9, 47, 57],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 36,  1,  0, 41,  7, 28, 58,
          28,  7, 41,  1, 47,  7, 52, 47, 53,  7, 42,  0,  6, 57],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [220]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()

        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)
    
# This is the model being used. It is using character embeddings
class CharEmbRNN(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, output_size):
        super(CharEmbRNN, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(emb_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = x.long()
        x = self.emb(x)
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

In [236]:
vocab_size = 66
hidden_size = 100
n_classes = 2
emb_size = 10
model = CharEmbRNN(vocab_size, emb_size, hidden_size, n_classes)

I've taken the functions from her last name classification model because that uses character-level embeddings as well. I'm using a naive version that just concats the question with the story and uses that as input into the RNN. I used that with regressing 2 outputs (start, end) and I got it to train but not sure how well it's actually doing. I think we correct way to do this is to treat it as a classification problem where each index of the story is a categroy and we use those predictions

In [237]:
def get_optimizer(model, lr = 0.1, wd = 0.0001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [238]:
def train(model, optim, train_dl):
    criterion = nn.MSELoss()
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
         # all zeros
        loss = 0
        x = torch.cat((x1, x2), 1)
        
        x = x.float()
        y = y.long()
        batch = x.shape[0]
        h = model.initHidden(batch)
        
        for t in range(x.shape[1]):
            #print(x[:,t])
            out, h = model(x[:,t], h)
            #max_h = np.maximum(h.detach().numpy(), max_h)
        
        print(out)
        print(y)
        #loss = F.cross_entropy(out, y)
        loss = criterion(out, y.float())
        optim.zero_grad()
        loss.backward()
        #loss.backward(retain_graph=True)
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

def train_loop(model, lr, train_dl, val_dl, epochs=20):
    optim = get_optimizer(model, lr =lr, wd = 0.0)
    for i in range(epochs):
        loss = train(model, optim, train_dl)
        val_loss, val_acc = val_metric(model, val_dl)
        if i%5 == 1: print("train loss %.3f val loss %.3f and val accuracy %.3f" % (loss, val_loss, val_acc))
            
def val_metric(model, val_dl):
    criterion = nn.MSELoss()
    model.eval()
    x1, x2, y = next(iter(val_dl))
    x = torch.cat((x1, x2), 1)
    x = x.float()
    y = y.long()
    N = x.shape[0]
    h = model.initHidden(N)
    for t in range(x.shape[1]):
        out, h = model(x[:,t], h)
    loss = criterion(out, y.float())
    _, pred = torch.max(out, 1)
    print(out)
    print(y)
    #acc = pred.eq(y).sum().float()/N
    return loss.item(), .22

In [239]:
optim = get_optimizer(model)

loss = train(model, optim, train_dl)

tensor([[-0.2625, -0.0864],
        [-0.3722, -0.1448],
        [-0.2625, -0.0864],
        [-0.1193, -0.0555],
        [-0.2695, -0.0642],
        [-0.2103, -0.0539],
        [-0.2512, -0.1471],
        [-0.0875, -0.0234]], grad_fn=<AddmmBackward>)
tensor([[   0,   16],
        [  22,   32],
        [ 122,  138],
        [ 488,  603],
        [1250, 1308],
        [ 614,  649],
        [1659, 1669],
        [2197, 2493]])
tensor([[ 0.6967,  0.5587],
        [ 1.0325,  0.7371],
        [-0.5078, -0.5837],
        [ 1.5960,  2.0563],
        [-0.6149, -0.4347],
        [ 2.2503,  2.0067],
        [ 1.0488,  0.7486],
        [ 1.8871,  2.0774]], grad_fn=<AddmmBackward>)
tensor([[ 221,  232],
        [ 688,  791],
        [1714, 1721],
        [ 346,  362],
        [ 567,  588],
        [5819, 5837],
        [ 439,  504],
        [ 470,  502]])
tensor([[ 6.7597,  7.0826],
        [ 8.4448,  8.2935],
        [-6.2398, -6.1299],
        [ 7.0281,  7.2302],
        [-5.8660, -6.1211],
      

In [230]:
train_loop(model, 0.01, train_dl, test_dl, epochs=100)

tensor([[ 49.6686,  49.1719],
        [ 49.9820,  49.4869],
        [ 49.9940,  49.4994],
        [-45.9037, -45.6669],
        [ 49.9866,  49.4915],
        [ 49.9820,  49.4869],
        [ 49.9820,  49.4869],
        [ 49.6834,  49.1869]], grad_fn=<AddmmBackward>)
tensor([[ 221,  232],
        [3798, 4072],
        [ 346,  362],
        [3171, 3186],
        [ 672,  695],
        [ 260,  270],
        [ 242,  310],
        [  22,   32]])
tensor([[48.7060, 48.2194],
        [50.8731, 50.3818],
        [50.8582, 50.3626],
        [50.8679, 50.3725],
        [49.9357, 49.4450],
        [49.4767, 49.0022],
        [50.8811, 50.3858],
        [50.8717, 50.3763]], grad_fn=<AddmmBackward>)
tensor([[1306, 1314],
        [ 488,  603],
        [  58,   93],
        [3859, 3877],
        [ 290,  474],
        [ 581,  658],
        [ 548,  551],
        [ 688,  791]])
tensor([[51.5992, 51.1154],
        [51.9000, 51.4174],
        [49.8441, 49.3521],
        [51.8996, 51.4170],
        [51.5704, 

tensor([[65.8734, 65.4576],
        [65.8488, 65.4401],
        [65.8734, 65.4576],
        [65.8489, 65.4402],
        [65.8734, 65.4576],
        [65.8488, 65.4401],
        [65.8734, 65.4576],
        [65.8734, 65.4576]], grad_fn=<AddmmBackward>)
tensor([[1250, 1308],
        [1603, 1668],
        [ 470,  502],
        [1714, 1721],
        [ 242,  310],
        [  28,   49],
        [2876, 2894],
        [1125, 1130]])
tensor([[66.7878, 66.3743],
        [66.7878, 66.3743],
        [66.7878, 66.3743],
        [66.7878, 66.3743],
        [66.7878, 66.3743],
        [66.7623, 66.3556],
        [66.7878, 66.3743],
        [66.7878, 66.3743]], grad_fn=<AddmmBackward>)
tensor([[ 614,  649],
        [3544, 3676],
        [ 260,  270],
        [ 122,  138],
        [3798, 4072],
        [1873, 1907],
        [5096, 5183],
        [3859, 3877]])
tensor([[ 67.7544,  67.3427],
        [-41.2813, -41.3490],
        [ 67.7544,  67.3427],
        [ 67.7544,  67.3427],
        [ 67.7544,  67.342

tensor([[82.5688, 82.2203],
        [82.7967, 82.4402],
        [82.7967, 82.4402],
        [82.7675, 82.4116],
        [82.7967, 82.4402],
        [82.7967, 82.4402],
        [79.9891, 79.7223],
        [82.7675, 82.4116]], grad_fn=<AddmmBackward>)
tensor([[2585, 2646],
        [ 294,  297],
        [ 465,  476],
        [1981, 1996],
        [ 252,  273],
        [1839, 1876],
        [ 644,  758],
        [ 499,  543]])
tensor([[82.7967, 82.4402],
        [82.5688, 82.2203],
        [82.7967, 82.4402],
        [82.7967, 82.4402],
        [82.7675, 82.4115],
        [82.7675, 82.4115],
        [82.7967, 82.4402],
        [82.7439, 82.3892]], grad_fn=<AddmmBackward>)
tensor([[ 178,  190],
        [ 567,  588],
        [1409, 1501],
        [1125, 1130],
        [1603, 1668],
        [  22,   32],
        [1490, 1498],
        [ 488,  603]])
tensor([[83.6961, 83.3444],
        [83.6961, 83.3444],
        [83.6961, 83.3444],
        [83.6961, 83.3444],
        [83.6961, 83.3444],
      

tensor([[97.7057, 97.4146],
        [97.7057, 97.4146],
        [97.7057, 97.4146],
        [97.7057, 97.4146],
        [97.1695, 96.8885],
        [97.7057, 97.4146],
        [97.7000, 97.4092],
        [97.7057, 97.4146]], grad_fn=<AddmmBackward>)
tensor([[ 122,  138],
        [1012, 1066],
        [1659, 1669],
        [ 555,  560],
        [ 581,  658],
        [ 178,  190],
        [  28,   49],
        [ 242,  310]])
tensor([[98.6239, 98.3355],
        [98.6187, 98.3306],
        [98.6187, 98.3306],
        [98.6185, 98.3303],
        [98.6239, 98.3355],
        [98.5152, 98.2306],
        [98.6187, 98.3306],
        [98.6239, 98.3355]], grad_fn=<AddmmBackward>)
tensor([[2601, 2606],
        [ 290,  474],
        [1603, 1668],
        [2197, 2493],
        [3802, 3830],
        [1306, 1314],
        [ 301,  324],
        [5819, 5837]])
tensor([[ 99.6056,  99.3198],
        [ 99.6010,  99.3154],
        [ 99.6010,  99.3154],
        [ 99.6056,  99.3198],
        [ 99.6010,  99.315

KeyboardInterrupt: 